## Kalman Filtering


In [1]:
import Simulator
import Observation
import Statistics
import ETKalmanFilter

### Definition of forward model

For Kalman filtering we have to be able to propagate a Gaussian distribution in time and to have access to the linear operators



In [2]:
timestamp = "2021_07_29-12_52_15"

grid, simulator = Simulator.from_file(timestamp)

### Definition of initial distribution 

The inital state is equipped with a Gaussian distribution specified by its mean and covariance. 

In [3]:
import numpy as np

def prior(grid, phi):
    """
    Constructing a Matern-type covariance prior with Matern-parameter phi
    where the matrix is normalized with the standard deviation
    """
    # INPUT! Bell mean
    mean = 0.0*np.ones(grid.N_x)
    # INPUT! Constant var
    var  = 1.0*np.ones(grid.N_x)

    # Matern covariance
    dist_mat = np.copy(grid.dist_mat)
    cov = (1+phi*dist_mat)*np.exp(-phi*dist_mat)
    
    # Normalisation
    var_mesh = np.meshgrid(np.sqrt(var),np.sqrt(var))[0]*np.meshgrid(np.sqrt(var),np.sqrt(var))[1]    

    return mean, var_mesh, cov

In [4]:
statistics = Statistics.Statistics(simulator, 500)
mean, var_mesh, cov = prior(grid=grid, phi=2.0)
statistics.set(mean, var_mesh*cov, nugget=0.5)
statistics.plot()

Please remember to set priors!


### Observations

In [ ]:
observation = Observation.from_file(simulator)

### Temporal evolution of model

In [ ]:
etkFilter = ETKalmanFilter.ETKalman(statistics, observation)
for t in range(observation.N_obs):
    print(t)
    statistics.propagate(25)
    etkFilter.filter(statistics.ensemble.ensemble, observation.obses[t])
    statistics.plot()